# Semantic Kernel

Semantic Kernel can **`automatically orchestrate` different plugins by using a `planner`**. With the planner, a user can ask your application to achieve a complex goal. For example, if you have a function that identifies which animal is in a picture and another function that tells knock-knock jokes, your user can say, “Tell me a knock-knock joke about the animal in the picture in this URL,” and the planner will automatically understand that it needs to call the identification function first and the “tell joke” function after it. **Semantic Kernel will automatically search and combine your plugins to achieve that goal and create a plan.** Then, Semantic Kernel will execute that plan and provide a response to the user:

<img src="../assets/semantic-kernel-architecture.png" />


**`LLM Cascade:`** The process of sending simpler requests to simpler models and complex requests to more complex models.

In [7]:
import semantic_kernel as sk

kernel = sk.Kernel()
print(help(request=kernel))

Help on Kernel in module semantic_kernel.kernel object:

class Kernel(semantic_kernel.filters.kernel_filters_extension.KernelFilterExtension, semantic_kernel.functions.kernel_function_extension.KernelFunctionExtension, semantic_kernel.services.kernel_services_extension.KernelServicesExtension, semantic_kernel.reliability.kernel_reliability_extension.KernelReliabilityExtension)
 |  Kernel(plugins: semantic_kernel.functions.kernel_plugin.KernelPlugin | dict[str, semantic_kernel.functions.kernel_plugin.KernelPlugin] | list[semantic_kernel.functions.kernel_plugin.KernelPlugin] | None = None, services: Union[~AI_SERVICE_CLIENT_TYPE, list[~AI_SERVICE_CLIENT_TYPE], dict[str, ~AI_SERVICE_CLIENT_TYPE], NoneType] = None, ai_service_selector: semantic_kernel.services.ai_service_selector.AIServiceSelector | None = None, *, retry_mechanism: semantic_kernel.reliability.retry_mechanism_base.RetryMechanismBase = None, function_invocation_filters: list[tuple[int, collections.abc.Callable[[~FILTER_CONTE

In [8]:
print(dir(kernel))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_validator__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_r

In [9]:
from dotenv import load_dotenv

load_dotenv()
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

gpt35 = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo")
gpt4o = OpenAIChatCompletion(ai_model_id="gpt-4o")

kernel.add_service(service=gpt35)
kernel.add_service(service=gpt4o)

To send the prompt to the service, we need to use a method called **`create_semantic_function`**.

### Running a simple prompt

1. **Load the prompt in a string variable.**

In [10]:
prompt = "Finish the following knock-knock joke. Knock, knock. Who's there? Dishes. Dishes who?"

2. Create a *`function`* by using the **`add_function`** method of the kernel. The *`function_name`* and *`plugin_name`* parameters of the function are required but are not used, so you can give your function and plugin whatever name you want.

In [11]:
prompt_function = kernel.add_function(
    function_name="subrata",
    plugin_name="sample",
    prompt=prompt,
)

3. Call the function. Note that all invocation methods are *`Asynchronous`*, so we need to use *`await`* to wait for their return.

In [12]:
response = await kernel.invoke(function=prompt_function, request=prompt)

print(response)

Function failed. Error: Error occurred while invoking function subrata: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"))
Something went wrong in function invocation. During function invocation: 'sample-subrata'. Error description: 'Error occurred while invoking function subrata: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more informat

KernelInvokeException: Error occurred while invoking function: 'sample-subrata'

In [13]:
# Entire Code in One Place
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion,
)

# Initialize the Kernel
kernel = sk.Kernel()

# Define Connectors
gpt35 = AzureChatCompletion(deployment_name="2024-08-01-preview")
gpt4o = OpenAIChatCompletion(ai_model_id="gpt-4o")

# Add the connectors to the kernel's service
kernel.add_service(service=gpt35)
kernel.add_service(service=gpt4o)

# Write a prompt
prompt = "Finish the following knock-knock joke. Knock, knock. Who's there? Dishes. Dishes who?"

# Create a function (semantic) using add_function method of the kernel
prompt_function = kernel.add_function(
    function_name="Subrata",
    plugin_name="Mondal",
    prompt=prompt,
)

# Invoke the created function
response = await kernel.invoke(
    function=prompt_function,
    request=prompt,
)

print(response)

Dishes a really bad joke!


## Semantic Functions and Native Functions

- **Semantic Functions:** are functions that connect to AI Services (LLM) to perform a task. The default parameter for all semantic functions is called *`input`*.

- **Native Functions:** are regular functions written in your programming language (Python).


The reason to differentiate a Native Function from any other Regular Function in your code is that the native funcion will have additional attributes that will tell the Kernel what it does.

After loading a native function into the Kernel, you can use it in chains that combine native and semantic functions. On top of it, **`Semantic Kernel Planner`** can use the function when creating a plan to achieve a user goal.

### Modified Semantic Function in Python

1. **Adding parameter to the Semantic Function**

In [14]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

args = KernelArguments(input="Boo")

response = await kernel.invoke(
    function=prompt_function,
    request=prompt,
    arguments=args,
)

print(response)

Dishes the police, open up!


### Create a Native Function

Native Functions are created in the same language our application is using like Python. 

In Python, the **Native functions need to be inside a `Class`**. The Class used to be called **`Skill`** and now it's called **`Plugin`**.

- **`Plugin:`** is just a *`collection of functions`*. You **cannot mix both the `Native` and `Semantic Function` in the same Plugin/Skill**. For, example we'll create a **`ShowManager` Plugin**.

**How to create a Native Function?**

To create a Native Function, we will use **`@kernel_function` decorator**. And the decorator must contain fields for **`description`** and **`name`**.

In [15]:
import random
from semantic_kernel.functions.kernel_function_decorator import kernel_function


# Plugin
class ShowManager:
    @kernel_function(
        description="Randomly choose among a theme for a joke.",
        name="random_theme",
    )
    def random_theme(self) -> str:
        themes: list[str] = ["Boo", "Dishes", "Art", "Needle", "Tank", "Police"]
        theme: str = random.choice(seq=themes)
        return theme

Now, to load the *Plugin (ShowManager)* and all its *functions* in the Kernel, we use the **`add_plugin`** method of the Kernel.

In [18]:
theme_choice_kernel_plugin = kernel.add_plugin(
    plugin=ShowManager(), plugin_name="ShowManager"
)

print(theme_choice_kernel_plugin)

name='ShowManager' description=None functions={'random_theme': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='random_theme', plugin_name='ShowManager', description='Randomly choose among a theme for a joke.', parameters=[], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x10e315490>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x10dbab140>, method=<bound method ShowManager.random_theme of <__main__.ShowManager object at 0x10e32fc50>>, stream_method=None)}


To call the Native Function from a *Plugin (ShowManager)*, simply put the name of the method within brackets.

In [19]:
response = await kernel.invoke(
    function=theme_choice_kernel_plugin["random_theme"],
)

print(response)

Tank


# Plugins

One of the greatest strengths of Microsoft Semantic Kernel is that you can create Semantic Plugins that are **Language Agnostic**. 

- **`Semantic Plugins:`** are **collections of Semantic Functions** that can be imported into the Kernel. It allows to separate your code from the AI Function which makes the code easier to maintain.

> Each Semantic Function of Semantic Plugin is defined by a directory containing two text files:
- **`config.json:`** contains the *`configuration`* for the semantic function like LLM, temperature, description and inputs.
- **`skprompt.txt:`** contains the *`prompt`* of the semantic function which will be sent to the LLM to generate response.

We are going to define a Plugin that contains two semantic functions. 
1. knock-knock joke generator function
2. function that take jokes as an input and explains why it's funny.

**Folder structure of Semantic Plugins:**

<img src="../assets/plugins-folder-structure.png"/>

## knock-knock joke semantic function

### `config.json` file

```json
{
    "schema": 1,
    "type": "completion",
    "description": "Generates a knock-knock joke based on user input",
    "default_services": [
        "gpt35",
        "gpt4o",
    ],
    "execution_settings": {
        "default": {
            "temperature": 0.8,
            "number_of_responses": 1,
            "top_p": 0.9,
            "max_tokens": 4000,
            "presence_penalty": 0.0,
            "frequency_penalty": 0.0,
        },
    },
    "input_variables": [
        {
            "name": "input",
            "description": "The topic that the Joke should be written about.",
            "required": true,
        },
    ],
}
```


- **`default_services:`** property is an arrat of preferred engines (LLM) to use (in order). Since knock-knock jokes are simple, we're going to use GPT-3.5 for it. All the parameters in the json file are required.

- **`description:`** field is important because it can be used by the **`Planner`**.

### `skprompt.txt` file

```plaintext
You are given a joke with the following setup:
Knock, knock!
Who's there?
{{$input}}!
{{$input}} who?
Repeat the whole setup and finish the joke with a funny punchline.
```

## Explain Joke Semantic Function

### `config.json` file

This file is almost exactly same as the config.json file used for the knock-knock joke function. We have made only 3 changes:

- The description
- The description of the *`input_variables`*
- The *`default_serices`* field

```json
{
    "schema": 1,
    "type": "completion",
    "description": "Give a joke, explain why it is funny.",
    "default_services": [
        "gpt4o",
    ],
    "execution_settings": {
        "default": {
            "temperature": 0.8,
            "number_of_responses": 1,
            "top_p": 0.9,
            "max_tokens": 4000,
            "presence_penalty": 0.0,
            "frequency_penalty": 0.0,
        },
    },
    "input_variables": [
        {
            "name": "input",
            "description": "The joke we want to explain.",
            "required": true,
        },
    ],
}
```

### `skprompt.txt` file

```plaintext
You are given the following joke:
{{$input}}
First, tell the joke.
Then, explain the joke
```

## Loading the Plugin using Python

You can load all the functions inside a plugin directory using the **`add_plugin`** method from the Kernel object. Just set the first parameter to *`None`* and set the *`parent_direcory`* parameter to the directory where the plugin is:

In [28]:
jokes_plugin = kernel.add_plugin(
    plugin=None,
    parent_directory="./plugins",
    plugin_name="jokes",
)

You can call the Functions in the same way as you would call a function from a Native Plugin by putting the function name within brackets:

In [ ]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

theme = "Scientist"
knock_knock_joke = await kernel.invoke(
    function=jokes_plugin["knock_knock_joke"],
    arguments=KernelArguments(input=theme),
)

print(knock_knock_joke)

Knock, knock!  
Who's there?  
Scientist!  
Scientist who?  

Scientist you a joke, but I wasn't sure if you'd react or remain inert!


Pass the result of Knock-Knock Joke to the *`explain_joke`* function.

In [34]:
joke_explanation = await kernel.invoke(
    function=jokes_plugin["explain_joke"],
    arguments=KernelArguments(input=theme),
)

print(joke_explanation)

Sure, here it is:

**Joke:**
Why can't you trust an atom?

Because they make up everything!

**Explanation:**
This joke plays on the double meaning of the phrase "make up." In science, atoms are the basic building blocks of matter, so they "make up" everything in the physical world. However, in everyday language, saying someone "makes up everything" means they are not truthful or they fabricate stories. The humor comes from this wordplay, as it initially suggests that atoms are untrustworthy, but the punchline reveals it’s because they literally compose all matter.


# Planner: Use it to run a Multistep Task.

Instead of calling functions yourself, let the Microsoft Semantic Kernel choose the functions for you. This makes the code simpler and give the users ability to combine your code in ways that you haven't considered.

- **Semantic Kernel has Two types of Plannner:**
    - *`Function Calling Stepwise Planner: (Deprecated)`* This planner employs a step-by-step approach, utilizing OpenAI's function calling capabilities to iteratively select and execute functions. It is particularly suitable for complex tasks requiring sequential function execution. However, with advancements in AI models, the Function Calling Stepwise Planner is being **`deprecated`** in favor of automatic function calling, which offers more streamlined and efficient task execution.

    - *`Handlebars planner: (Deprecated)`* This planner uses Handlebars syntax to generate plans, allowing the model to leverage native features like loops and conditions without additional prompting. It enables the creation of customized plans by defining templates that the AI can follow. However, the Handlebars Planner is also being *`deprecated`* in favor of more advanced planning methods that provide greater reliability and efficiency.